# ResNet101
upload transposed2.Rds

In [6]:
import rpy2.robjects as ro
from rpy2.robjects import r
import torch
from rpy2.robjects import pandas2ri
from torch.utils.data import Dataset, DataLoader
from torchvision import models
from torch import nn
from rpy2.robjects.conversion import localconverter

torch.manual_seed(88888888)

DATA_PATH = 'transposed2.Rds' #'/home/jgburk/PycharmProjects/IntroductionToDeepLearning/transposed.Rds'
NUM_FEATURES = 1000
NUM_CLASSES = 7
NUM_TRAIN_IT = 1000

BATCH_SIZE = 32


class PandasDataset(Dataset):

    def __init__(self, rds_fn):
      with localconverter(ro.default_converter + pandas2ri.converter):
        df = ro.conversion.rpy2py(r.readRDS(rds_fn))

        features = df.iloc[:, 0:NUM_FEATURES].values
        targets = df['sample_type'].values

        self.x = torch.tensor(features)
        self.y = torch.tensor(targets - 1)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, i):
        return self.x[i], self.y[i]


def train(loader, dv):
    model.train()

    correct = 0
    for batch in loader:  # Iterate in batches over the training dataset.
        x = batch[0].reshape(BATCH_SIZE, 10, 10, 10).to(dv)
        y = batch[1].to(dv)
        out = model(x.float())  # Perform a single forward pass.
        y = torch.squeeze(y)
        loss = criterion(out, y.long())  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        correct += int((pred == y).sum())  # Check against ground-truth labels.
    return correct / len(loader.dataset)  # Derive ratio of correct predictions.


rds_dataset = PandasDataset(DATA_PATH)
training_loader = DataLoader(rds_dataset, batch_size=BATCH_SIZE, shuffle=False)

model = models.resnet101(num_classes=NUM_CLASSES) # resnet with 101 layers
conv1 = model.conv1
model.conv1 = nn.Conv2d(10,
                        conv1.out_channels,
                        conv1.kernel_size,
                        conv1.stride,
                        conv1.padding,
                        conv1.dilation,
                        conv1.groups,
                        conv1.bias)
device = torch.device('cuda')
model.to(device)
optimizer = torch.optim.AdamW(model.parameters())
criterion = torch.nn.CrossEntropyLoss()

for epoch in range(NUM_TRAIN_IT):
    train(training_loader, device)
    train_acc = train(training_loader, device)
    print(f'Epoch: {epoch}, Train Acc: {train_acc}')
    if train_acc == 1.0:
        break

Epoch: 0, Train Acc: 0.06944444444444445
Epoch: 1, Train Acc: 0.1597222222222222
Epoch: 2, Train Acc: 0.3159722222222222
Epoch: 3, Train Acc: 0.4513888888888889
Epoch: 4, Train Acc: 0.3958333333333333
Epoch: 5, Train Acc: 0.5381944444444444
Epoch: 6, Train Acc: 0.5659722222222222
Epoch: 7, Train Acc: 0.65625
Epoch: 8, Train Acc: 0.6979166666666666
Epoch: 9, Train Acc: 0.6909722222222222
Epoch: 10, Train Acc: 0.8576388888888888
Epoch: 11, Train Acc: 0.9305555555555556
Epoch: 12, Train Acc: 0.8576388888888888
Epoch: 13, Train Acc: 0.6805555555555556
Epoch: 14, Train Acc: 0.2743055555555556
Epoch: 15, Train Acc: 0.7430555555555556
Epoch: 16, Train Acc: 0.9340277777777778
Epoch: 17, Train Acc: 0.9027777777777778
Epoch: 18, Train Acc: 0.8993055555555556
Epoch: 19, Train Acc: 0.8784722222222222
Epoch: 20, Train Acc: 0.9097222222222222
Epoch: 21, Train Acc: 0.8993055555555556
Epoch: 22, Train Acc: 0.9270833333333334
Epoch: 23, Train Acc: 0.9548611111111112
Epoch: 24, Train Acc: 0.920138888888